# Optymalizacja masy

In [2]:
using JuMP
using CSV
using DataFrames
using AmplNLWriter

### Prepare input data
data_mass = dropmissing(CSV.read("input_data/plecak.csv"))
data_plan = CSV.read("input_data/plan.csv")

plan_summary = by(data_plan, [:Przedmiot], plan_summary -> size(plan_summary, 1))
mass_summary = by(data_mass, [:Przedmiot], mass_summary -> sum(mass_summary[:Waga]))

przybory_mass = mass_summary[mass_summary[:Przedmiot] .== "Przybory", :][:x1] / 1e3

mass_summary = mass_summary[mass_summary[:Przedmiot] .!= "Przybory", :]

dict_plan = Dict(zip(plan_summary[:Przedmiot], plan_summary[:x1]))
dict_mass = Dict(zip(mass_summary[:Przedmiot], mass_summary[:x1]))

data_in= DataFrame(Przedmiot = String[],
    Sztuk = Int64[],
    Waga = Float64[])

for row in 1:size(plan_summary, 1)
    course = plan_summary[:Przedmiot][row]
    push!(data_in,[course, get(dict_plan, course, 0), get(dict_mass, course, 0) / 1e3])
end
println(data_in)
### set parameters
course_no = size(data_in,1)
max_course_occurrences = maximum(data_in[:Sztuk])
days = 5
max_per_day = 8
min_per_day = 3


# avg mass for comparison
avg_mass = sum(data_in[:Sztuk].*data_in[:Waga]) / days + przybory_mass

# objective scaling factor - workaround to set solution tolerance
objective_scaling_factor = 20e-3
data_in[:Waga] = data_in[:Waga] * objective_scaling_factor

# dictionaries for results
course_map = Dict(zip(1:course_no, data_in[:Przedmiot]))
mass_map = Dict(zip(1:course_no, data_in[:Waga]))

# maximum number of a course occurences per day
max_occurences_per_day  = 2
if max_occurences_per_day < max_course_occurrences
    max_course_occurrences = max_occurences_per_day
end



print("Masa przyborów: ")
println(przybory_mass[1])

11×3 DataFrames.DataFrame
│ Row │ Przedmiot   │ Sztuk │ Waga  │
├─────┼─────────────┼───────┼───────┤
│ 1   │ Matematyka  │ 4     │ 0.697 │
│ 2   │ Angielski   │ 3     │ 0.715 │
│ 3   │ Polski      │ 5     │ 1.676 │
│ 4   │ WF          │ 3     │ 0.297 │
│ 5   │ Religia     │ 2     │ 0.568 │
│ 6   │ Przyroda    │ 2     │ 1.214 │
│ 7   │ Technika    │ 1     │ 0.756 │
│ 8   │ Informatyka │ 1     │ 0.159 │
│ 9   │ Plastyka    │ 1     │ 0.167 │
│ 10  │ Historia    │ 1     │ 0.484 │
│ 11  │ Muzyka      │ 1     │ 0.36  │
Masa przyborów: 1.415


In [5]:
### Opt start, define model, variables, constraints and objective
# Solver engine can be found on from https://ampl.com/products/solvers/open-source/ 
solver_couenne="couenne-win64/couenne.exe"
mdl=Model(solver=AmplNLSolver(solver_couenne,["tol=1"]))

@variable(mdl, x[1:days,1:course_no, 1:max_course_occurrences], Bin, start=1)
@variable(mdl, day[1:days])

for d in 1:days
    
    @constraint(mdl, day[d] == sum(sum(x[d,c,m]*data_in[:Waga][c] 
                for c in 1:course_no) 
                for m in 1:max_course_occurrences))
    
#     @constraint(mdl, sum(sum(x[d,c,m]*m 
#                 for c in 1:course_no) 
#                 for m in 1:max_course_occurrences) <= max_per_day)

#     @constraint(mdl, sum(sum(x[d,c,m]*m 
#                 for c in 1:course_no) 
#                 for m in 1:max_course_occurrences) >= min_per_day) 
    
    for c in 1:course_no
            @constraint(mdl, sum(x[d,c,m]*m 
                    for m in 1:max_course_occurrences) <= max_course_occurrences)
    end
end

for c in 1:course_no
    @constraint(mdl, sum(sum(x[d,c,m]*m 
                for d in 1:days) 
                for m in 1:max_course_occurrences) == data_in[:Sztuk][c])
end

@NLobjective(mdl,Min, sum((day[d])^2 for d in 1:days))

print(mdl)

Min day[1] ^ 2.0 + day[2] ^ 2.0 + day[3] ^ 2.0 + day[4] ^ 2.0 + day[5] ^ 2.0
Subject to
 day[1] - 0.01394 x[1,1,1] - 0.0143 x[1,2,1] - 0.03352 x[1,3,1] - 0.00594 x[1,4,1] - 0.011359999999999999 x[1,5,1] - 0.02428 x[1,6,1] - 0.01512 x[1,7,1] - 0.00318 x[1,8,1] - 0.00334 x[1,9,1] - 0.00968 x[1,10,1] - 0.0072 x[1,11,1] - 0.01394 x[1,1,2] - 0.0143 x[1,2,2] - 0.03352 x[1,3,2] - 0.00594 x[1,4,2] - 0.011359999999999999 x[1,5,2] - 0.02428 x[1,6,2] - 0.01512 x[1,7,2] - 0.00318 x[1,8,2] - 0.00334 x[1,9,2] - 0.00968 x[1,10,2] - 0.0072 x[1,11,2] == 0
 x[1,1,1] + 2 x[1,1,2] <= 2
 x[1,2,1] + 2 x[1,2,2] <= 2
 x[1,3,1] + 2 x[1,3,2] <= 2
 x[1,4,1] + 2 x[1,4,2] <= 2
 x[1,5,1] + 2 x[1,5,2] <= 2
 x[1,6,1] + 2 x[1,6,2] <= 2
 x[1,7,1] + 2 x[1,7,2] <= 2
 x[1,8,1] + 2 x[1,8,2] <= 2
 x[1,9,1] + 2 x[1,9,2] <= 2
 x[1,10,1] + 2 x[1,10,2] <= 2
 x[1,11,1] + 2 x[1,11,2] <= 2
 day[2] - 0.01394 x[2,1,1] - 0.0143 x[2,2,1] - 0.03352 x[2,3,1] - 0.00594 x[2,4,1] - 0.011359999999999999 x[2,5,1] - 0.02428 x[2,6,1] - 0.01512

In [1]:
# solve
status = solve(mdl)

In [6]:
### print and save results
day_of_week=Dict(zip([1, 2, 3, 4, 5], ["Poniedzialek", "Wtorek", "Sroda", "Czwartek", "Piatek"]))

plan = DataFrame([String,String,String,String,String],
    [:Poniedzialek,:Wtorek,:Sroda,:Czwartek,:Piatek], 10)
sum_week = 0

for d in 1:days
    i=1
    plan[i,d] = string(getvalue(day[d]) / objective_scaling_factor + przybory_mass)
    sum_week += getvalue(day[d]) / objective_scaling_factor + przybory_mass
    i+=1
    for c in 1:course_no
        for m in 1:max_course_occurrences
            value = getvalue(x[d,c,m])*m
            if abs(value-0.0)>1e-3              
                for repeat in 1:m
                    plan[i,d] = get(course_map, c, "brak kursu w slowniku")
                    i+=1
                end
            end
        end
    end
    for rest in i:10
         plan[rest,d] = ""
    end
end

max_mass = maximum(getvalue(day)) / objective_scaling_factor + przybory_mass


println("masa maxymalna")
println(max_mass)
println("masa sumaryczna")
println(sum_week)
println("masa srednia bez optim")
println(avg_mass)
print(plan)

CSV.write("results/plan_wynik_min.csv",plan)

masa maxymalna
[3.907]
masa sumaryczna
[19.229]
masa srednia bez optim
[5.3538]
10×5 DataFrames.DataFrame
│ Row │ Poniedzialek │ Wtorek    │ Sroda      │ Czwartek   │ Piatek   │
├─────┼──────────────┼───────────┼────────────┼────────────┼──────────┤
│ 1   │ [3.907]      │ [3.806]   │ [3.788]    │ [3.859]    │ [3.869]  │
│ 2   │ Polski       │ Angielski │ Matematyka │ Matematyka │ Przyroda │
│ 3   │ Polski       │ Polski    │ Matematyka │ Matematyka │ Przyroda │
│ 4   │ WF           │ Polski    │ Polski     │ Angielski  │ Technika │
│ 5   │ WF           │           │            │ Angielski  │ Historia │
│ 6   │ Informatyka  │           │            │ WF         │          │
│ 7   │ Muzyka       │           │            │ Religia    │          │
│ 8   │              │           │            │ Religia    │          │
│ 9   │              │           │            │ Plastyka   │          │
│ 10  │              │           │            │            │          │

CSV.Sink{Void,DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        missingstring: ""
        dateformat: nothing
        decimal: '.'
        truestring: 'true'
        falsestring: 'false'
        internstrings: true, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "results/plan_wynik_min.csv", 42, true, String["Poniedzialek", "Wtorek", "Sroda", "Czwartek", "Piatek"], 5, false, Val{false})